# DataLake to SQL: 2021 Air Quality USA by County

In [0]:
# imports
from pyspark.sql.functions import isnan

In [0]:
# Create a mount point
# Creating a mount point to write to

storageAccount = "gen10datafund2111"
storageContainer = "jadr-health-insights"
clientSecret = dbutils.secrets.get(scope = "jadr_blob", key = "clientSecret")
clientid = dbutils.secrets.get(scope = "jadr_blob", key = "clientid")
mount_point="/mnt/jadr"

configs = {"fs.azure.account.auth.type": "OAuth",
   "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
   "fs.azure.account.oauth2.client.id": clientid,
   "fs.azure.account.oauth2.client.secret": clientSecret, 
   "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/d46b54b2-a652-420b-aa5a-2ef7f8fc706e/oauth2/token",
   "fs.azure.createRemoteFileSystemDuringInitialization": "true"}

try:
    dbutils.fs.unmount(mount_point)
except:
    pass

dbutils.fs.mount(source = "abfss://"+storageContainer+"@"+storageAccount+".dfs.core.windows.net/", mount_point = mount_point, extra_configs = configs)
display(dbutils.fs.ls("/mnt/jadr"))

/mnt/jadr has been unmounted.

path,name,size,modificationTime
dbfs:/mnt/jadr/Data/,Data/,0,1643742636000
dbfs:/mnt/jadr/ML-Models/,ML-Models/,0,1643906451000
dbfs:/mnt/jadr/deleteme.txt,deleteme.txt,8,1643742578000


In [0]:
# Read in the data and drop duplicates
df = spark.read.options(header=True).json('/mnt/jadr/Data/aqi_stream/*.json')
df = df.distinct()
display(df)
print(df.count())

Lead1stMax,Lead2ndMax,Lead99perc,LeadMean,LeadMethod,LeadMetric,LeadStd,LeadUnits,NO21stMax,NO22ndMax,NO299perc,NO2Mean,NO2Method,NO2Metric,NO2Std,NO2Units,Ozone1stMax,Ozone2ndMax,Ozone99perc,OzoneMean,OzoneMethod,OzoneMetric,OzoneStd,OzoneUnits,PM101stMax,PM102ndMax,PM1099perc,PM10Mean,PM10Method,PM10Metric,PM10Std,PM10Units,PM251stMax,PM252ndMax,PM2599perc,PM25Mean,PM25Method,PM25Metric,PM25Std,PM25Units,SO21stMax,SO22ndMax,SO299perc,SO2Mean,SO2Method,SO2Metric,SO2Std,SO2Units,county,state,year
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.074,0.074,0.072,0.048311,INSTRUMENTAL - ULTRA VIOLET,Daily maxima of observed hourly values (between 9:00 AM and 8:00 PM),0.010726,Parts per million,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,Rowan,North Carolina,2021
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.081,0.076,0.073,0.046301,INSTRUMENTAL - ULTRA VIOLET ABSORPTION,Daily maxima of observed hourly values (between 9:00 AM and 8:00 PM),0.010837,Parts per million,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,Glenn,California,2021
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.08,0.075,0.07,0.047634,INSTRUMENTAL - ULTRA VIOLET ABSORPTION,Daily maxima of observed hourly values (between 9:00 AM and 8:00 PM),0.010553,Parts per million,null,null,null,null,null,null,null,null,54.95,52.25,31.85,9.355924,Multiple Methods Used,Observed Values,5.943445,Micrograms/cubic meter (LC),null,null,null,null,null,null,null,null,Rock Island,Illinois,2021
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.074,0.072,0.07,0.044342,INSTRUMENTAL - ULTRA VIOLET,Daily maxima of observed hourly values (between 9:00 AM and 8:00 PM),0.010735,Parts per million,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,Wexford,Michigan,2021
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0745,0.0695,0.0675,0.051236000000000004,INSTRUMENTAL - ULTRA VIOLET ABSORPTION,Daily maxima of observed hourly values (between 9:00 AM and 8:00 PM),0.0072085,Parts per million,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,Coconino,Arizona,2021
null,null,null,null,null,null,null,null,26.03333333333333,19.599999999999998,19.833333333333332,6.977667,Multiple Methods Used,Daily Maximum 1-hour average,3.018803,Parts per billion,0.09966666666666667,0.07533333333333332,0.07366666666666666,0.05262633333333333,INSTRUMENTAL - ULTRA VIOLET,Daily maxima of observed hourly values (between 9:00 AM and 8:00 PM),0.009252,Parts per million,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,La Plata,Colorado,2021
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0735,0.071,0.0695,0.045564999999999994,INSTRUMENTAL - ULTRA VIOLET ABSORPTION,Daily maxima of observed hourly values (between 9:00 AM and 8:00 PM),0.0099255,Parts per million,null,null,null,null,null,null,null,null,145.7,50.35,31.85,9.797997,Multiple Methods Used,Daily Mean,6.264309,Micrograms/cubic meter (LC),null,null,null,null,null,null,null,null,Peoria,Illinois,2021
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.085,0.073,0.072,0.048477,Multiple Methods Used,Daily maxima of observed hourly values (between 9:00 AM and 8:00 PM),0.009936,Parts per million,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,Madison,Indiana,2021
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.113,0.099,0.098,0.047567,INSTRUMENTAL - ULTRA VIOLET ABSORPTION,Daily maxima of obse

775

In [0]:
# SQL Database Connection
server = 'gen10-data-fundamentals-21-11-sql-server.database.windows.net'
database = 'jadr-SQL-Database'
port = '1433'
user = dbutils.secrets.get(scope = "jadr_blob", key = "SQLUser_dg")
password = dbutils.secrets.get(scope = "jadr_blob", key = "SQLPassword_dg")
url = f"jdbc:sqlserver://{server}:{port};databaseName={database};user={user};password={password};" 

## Convert to Foreign Keys

In [0]:
### Read in the database County, State, Method, Metric, Unit tables
jdbcDF_state = spark.read.format("jdbc") \
    .option("url", url) \
    .option("dbtable", "State") \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .load()
jdbcDF_county = spark.read.format("jdbc") \
    .option("url", url) \
    .option("dbtable", "County") \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .load()
jdbcDF_method = spark.read.format("jdbc") \
    .option("url", url) \
    .option("dbtable", "Method") \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .load()
jdbcDF_metric = spark.read.format("jdbc") \
    .option("url", url) \
    .option("dbtable", "Metric") \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .load()
jdbcDF_unit = spark.read.format("jdbc") \
    .option("url", url) \
    .option("dbtable", "Unit") \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .load()

# Convert null to "" for future joining
jdbcDF_method = jdbcDF_method.fillna("")
jdbcDF_metric = jdbcDF_metric.fillna("")
jdbcDF_unit = jdbcDF_unit.fillna("")

In [0]:
### Change state to STATE_ID

# Join on state name
df_join = df.join(jdbcDF_state, df.state == jdbcDF_state.STATE_NAME , "leftouter")

# Write state to database if it doesn't yet exist
if df_join.filter(df_join.STATE_ID.isNull()).count() > 0:
    table = "State"
    df_toload = df_join.filter(df_join.STATE_ID.isNull()).select("state").withColumnRenamed("state","STATE_NAME").distinct()
    
    # Write the new state to the database
    df_toload.write.format("jdbc") \
        .option("url", url) \
        .mode("append") \
        .option("dbtable", table) \
        .option("user", user) \
        .option("password", password) \
        .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
        .save()

    # Call the database again
    jdbcDF_state = spark.read.format("jdbc") \
        .option("url", url) \
        .option("dbtable", "State") \
        .option("user", user) \
        .option("password", password) \
        .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
        .load()
    
    # Join on state name again
    df_join = df.join(jdbcDF_state, df.state == jdbcDF_state.STATE_NAME , "leftouter") # Join the database again

# Drop unnecessary columns
df = df_join.drop("state","STATE_ABBR","STATE_NAME")

In [0]:
### Change county and STATE_ID to COUNTY_ID

# Join on county name and stateID. Drop the duplicate STATE_ID (not sure why it does that...)
df_join = df.join(jdbcDF_county, (df.county == jdbcDF_county.COUNTY_NAME) & (df.STATE_ID == jdbcDF_county.STATE_ID), "leftouter").drop(jdbcDF_county["STATE_ID"])

# Write county to database if it doesn't yet exist
if df_join.filter(df_join.COUNTY_ID.isNull()).count() > 0:
    table = "County"
    df_toload = df_join.filter(df_join.COUNTY_ID.isNull()).select("STATE_ID","county").withColumnRenamed("county","COUNTY_NAME").distinct()
    
    # Write the new state to the database
    df_toload.write.format("jdbc") \
        .option("url", url) \
        .mode("append") \
        .option("dbtable", table) \
        .option("user", user) \
        .option("password", password) \
        .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
        .save()
    
    # Call the database again
    jdbcDF_county = spark.read.format("jdbc") \
        .option("url", url) \
        .option("dbtable", "County") \
        .option("user", user) \
        .option("password", password) \
        .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
        .load()
    
    # Join on state name again
    df_join = df.join(jdbcDF_county, (df.county == jdbcDF_county.COUNTY_NAME) & (df.STATE_ID == jdbcDF_county.STATE_ID), "leftouter").drop(jdbcDF_county["STATE_ID"]) # Join the database again

# Drop unnecessary columns
df = df_join.drop("county","STATE_ID","COUNTY_NAME")

In [0]:
# Change methods to method_ID
pols=["Lead","NO2","Ozone","PM10","PM25","SO2"]
for pol in pols:
    
    # null =/= null with a join--replace with empty str literal
    df = df.fillna("",subset=f"{pol}Method")
    
    # Join on method name
    df_join = df.join(jdbcDF_method, df[f"{pol}Method"] == jdbcDF_method.METHOD_NAME , "leftouter")
    
    # Write method to database if it doesn't yet exist
    if df_join.filter(df_join.METHOD_ID.isNull()).count() > 0:
        table = "Method"
        df_toload = df_join.filter(df_join.METHOD_ID.isNull()).select(f"{pol}Method").withColumnRenamed(f"{pol}Method","METHOD_NAME").distinct().replace("",None)
        display(df_toload)
        # Write the new method(s) to the database
        df_toload.write.format("jdbc") \
            .option("url", url) \
            .mode("append") \
            .option("dbtable", table) \
            .option("user", user) \
            .option("password", password) \
            .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .save()

        # Call the database again
        jdbcDF_method = spark.read.format("jdbc") \
            .option("url", url) \
            .option("dbtable", "Method") \
            .option("user", user) \
            .option("password", password) \
            .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .load()
        jdbcDF_method = jdbcDF_method.fillna("")
        
        # Join on method name again
        df_join = df.join(jdbcDF_method, df[f"{pol}Method"] == jdbcDF_method.METHOD_NAME , "leftouter") # Join the database again

    # Drop unnecessary columns
    df = df_join.drop(f"{pol}Method","METHOD_NAME").withColumnRenamed("METHOD_ID",f"{pol.upper()}_METHOD_ID")

In [0]:
#Change metric to metric_ID
pols=["Lead","NO2","Ozone","PM10","PM25","SO2"]
for pol in pols:
    
    # null =/= null with a join--replace with empty str literal
    df = df.fillna("",subset=f"{pol}Metric")
    
    # Join on metric name
    df_join = df.join(jdbcDF_metric, df[f"{pol}Metric"] == jdbcDF_metric.METRIC_NAME , "leftouter")
    
    # Write metric to database if it doesn't yet exist
    if df_join.filter(df_join.METRIC_ID.isNull()).count() > 0:
        table = "Metric"
        df_toload = df_join.filter(df_join.METRIC_ID.isNull()).select(f"{pol}Metric").withColumnRenamed(f"{pol}Metric","METRIC_NAME").distinct().replace("",None)
        # Write the new metric(s) to the database
        df_toload.write.format("jdbc") \
            .option("url", url) \
            .mode("append") \
            .option("dbtable", table) \
            .option("user", user) \
            .option("password", password) \
            .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .save()

        # Call the database again
        jdbcDF_metric = spark.read.format("jdbc") \
            .option("url", url) \
            .option("dbtable", "Metric") \
            .option("user", user) \
            .option("password", password) \
            .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .load()
        jdbcDF_metric = jdbcDF_metric.fillna("")
        
        # Join on metric name again
        df_join = df.join(jdbcDF_metric, df[f"{pol}Metric"] == jdbcDF_metric.METRIC_NAME , "leftouter") # Join the database again

    # Drop unnecessary columns
    df = df_join.drop(f"{pol}Metric","METRIC_NAME").withColumnRenamed("METRIC_ID",f"{pol.upper()}_METRIC_ID")

In [0]:
# Change units to units_ID
pols=["Lead","NO2","Ozone","PM10","PM25","SO2"]
for pol in pols:
    
    # null =/= null with a join--replace with empty str literal
    df = df.fillna("",subset=f"{pol}Units")
    
    # Join on unit name
    df_join = df.join(jdbcDF_unit, df[f"{pol}Units"] == jdbcDF_unit.UNIT_NAME , "leftouter")

    # Write unit to database if it doesn't yet exist
    if df_join.filter(df_join.UNIT_ID.isNull()).count() > 0:
        table = "Unit"
        df_toload = df_join.filter(df_join.UNIT_ID.isNull()).select(f"{pol}Units").withColumnRenamed(f"{pol}Units","UNIT_NAME").distinct().replace("",None)
        display(df_toload)
        # Write the new unit(s) to the database
        df_toload.write.format("jdbc") \
            .option("url", url) \
            .mode("append") \
            .option("dbtable", table) \
            .option("user", user) \
            .option("password", password) \
            .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .save()

        # Call the database again
        jdbcDF_unit = spark.read.format("jdbc") \
            .option("url", url) \
            .option("dbtable", "Unit") \
            .option("user", user) \
            .option("password", password) \
            .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .load()
        jdbcDF_unit = jdbcDF_unit.fillna("")
        
        # Join on unit name again
        df_join = df.join(jdbcDF_unit, df[f"{pol}Units"] == jdbcDF_unit.UNIT_NAME , "leftouter") # Join the database again

    # Drop unnecessary columns
    df = df_join.drop(f"{pol}Units","UNIT_NAME").withColumnRenamed("UNIT_ID",f"{pol.upper()}_UNITS_ID")

## Change column names to match SQL Database

In [0]:
# Copy schema from database and make DataFrame with that schema

# Load in database
df_loaded = spark.read.format("jdbc") \
    .option("url", url) \
    .option("dbtable", "AirQualityDataCounty") \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .load()
df_loaded = df_loaded.drop("AQ_ID") # We don't need the IDs

# Reorder columns to match order from database
df = df.select("COUNTY_ID", "year",
                             "LeadMean", "Lead1stMax", "Lead99perc", "LeadStd",
                             "Lead2ndMax","LEAD_METHOD_ID","LEAD_METRIC_ID","LEAD_UNITS_ID",
                             "NO2Mean", "NO21stMax", "NO299perc", "NO2Std",
                             "NO22ndMax", "NO2_METHOD_ID","NO2_METRIC_ID","NO2_UNITS_ID",
                             "OzoneMean", "Ozone1stMax", "Ozone99perc", "OzoneStd",
                             "Ozone2ndMax", "OZONE_METHOD_ID","OZONE_METRIC_ID","OZONE_UNITS_ID", 
                             "PM10Mean", "PM101stMax", "PM1099perc", "PM10Std",
                             "PM102ndMax", "PM10_METHOD_ID","PM10_METRIC_ID","PM10_UNITS_ID",
                             "PM25Mean", "PM251stMax", "PM2599perc", "PM25Std", 
                             "PM252ndMax", "PM25_METHOD_ID","PM25_METRIC_ID","PM25_UNITS_ID",
                             "SO2Mean", "SO21stMax", "SO299perc", "SO2Std",
                             "SO22ndMax", "SO2_METHOD_ID","SO2_METRIC_ID","SO2_UNITS_ID")

#Replace schema
df = sqlContext.createDataFrame(df.rdd, df_loaded.schema)

## Upload to SQL Database

In [0]:
### Upload the Dataframe to the SQL database
table = "AirQualityDataCounty"

# Subtract the two to only leave what is not in the database. 
df_toload = df.dropDuplicates(subset=["COUNTY_ID","YEAR"]).subtract(df_loaded)

# Upload all the data that wasn't already in the database
df_toload.write.format("jdbc").option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .mode("append") \
    .option("dbtable", table) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .save()

In [0]:
# This shows which rows are being repeated due to duplicates from the consumer to datalake, possibly related to issues from producer to consumer. Worth investigating further if time allots.
df_copies = df.groupBy(["COUNTY_ID","YEAR"]).count().filter("count > 1")
display(df_copies)

COUNTY_ID,YEAR,count
1615,2021,2
2343,2021,2
54,2021,2
3118,2021,2
1983,2021,2
473,2021,2
2238,2021,2
118,2021,2
586,2021,2
